<h1 align="center">  Exploratory Data Analysis (EDA) notebook </h1>

#### Author:
Wenwen Kong

#### Updated:
Nov 11, 2022

---

### Details of our EDA

- #### Deal with the missing value in the climate variables 

    - Note that we've fixed the missing values of soybean yield in the data ingestion notebook
    
    - Here, we deal with missing values in the NLDAS dataset. We remove features that have too many missing values and fill in missing values using the forward interpolation method. 

- #### Feature selection principles
    - Typical soybean season spans May through October, so we won't consider environmental effects during months (i.e., November and December) after the soybean season.

    - We only include features that have a possible predictive ability soybean yield, based on both our domain knowledge and the magnitude of correlation coefficient between yield and the feature.
    
    - We want to remove redundant features to avoid overfitting. Especially, some climate variables are highly autocorrelated across months, in those cases it's not necessary to keep all months and we do aggregations (i.e., seasonal average).
    
    - Nonlinear relationship between some climate variable and annual soybean yield is possible. In those cases, we keep the feature despite a weak linear correlation coefficient. 
    
    - We also do feature engineering when necessary. In this specific work, we derived evaporative fraction using surface energy fluxes, and decided to use the evaporative fraction (instead of latent and sensible heat fluxes) as a feature. 
    
    - Besides the current year's climate, we also consider whether past year's climate matters. We explored past year's annual total rainfall, and past winter's snowfall, rainfall, and soil moisture. We decided to include the past year's annual total rainfall as another feature to predict current year's annual yield. 

- #### Output - two .csv files 
 
    - The first .csv file will be a raw yield and feature dataframe, with all missing value issues fixed and unwanted months (November and December) removed. See [here](#save_raw_feature) for the saved dataframe. 
    
    - The second .csv file will be an intermediate yield and selected feature dataframe, in which only features that I think are important are included. See [here](#save_selected_feature) for the save dataframe.
    
    - Both .csv files will be further tested separately during our modeling step. Based on the feature importance findings from the modeling step, we will interate the feature set and converge to a final version. 


### Caveats

- #### Spatial variation 

Great Plains likely behave differently from other areas in terms of the soybean yield controlling factors. Currently we view all regions interchangable, which may weaken the predictive power of our model. For now we stick to our assumption and keep Great Plains region in the dataset.

- #### Feature selection methods 

Feature selection in the current notebook is mainly based on correlation matrix and domain knowledge. As [this post](https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2) mentioned, feature selection methods fall into three categories: 

   - `filter based` (correlation or chi-square) 
    
   - `wrapper based` ([recursive feature elimination](https://machinelearningmastery.com/rfe-feature-selection-in-python/))
    
   - `embedded` (algorithms that have built-in feature selection methods, such as Lasso and Random Forest, which have their feature selection methods)

It's likely that I've missed some important features or included some not that important features. During the model trainning step, we will check our feature importance once again. Particularly, I will explore whether there data-driven features that I can include to update my feature set. 


<a id='section_0'></a>
---
## Table of contents 


### 1. [Background](#section_1)
    
    
  - #### 1.1 [U.S. Soybean season](#section_1.1)

  - #### 1.2 [Literature findings](#section_1.2) 
    
### 2. [Import packages and data](#section_2)

  - #### 2.1 [Import packages](#section_2.1)
    
  - #### 2.2 [Read the .csv file generated from data ingestion notebook](#section_2.2)
  
### 3. [EDA of the yield data](#section_3)  

  - #### 3.1 [Distribution of annual yield from 1981 to 2016](#section_3.1)
  
  - #### 3.2 [Boxplot of annual yield from 1981 to 2016](#section_3.2)
  
  - #### 3.3 [Climatology, trend, and interannual variability](#section_3.3)
  
### 4. [EDA of the climate data ](#section_4)

  - #### 4.1 [Get familar with climate variables and their standard names](#section_4.1)
  
  - #### 4.2 [Deal with the missing values](#section_4.2)

### 5. [Feature selection](#section_5)
  
  - #### 5.1 [Drop current Nov-Dec columns and streamflow data](#section_5.1)
  
  - #### 5.2 [Exploring the correlation between yield and current monthly climate](#section_5.2)
   
       - 5.2a [Suface energy flux variables](#section_5.2a)
       
       - 5.2b [Surface water flux / storage variables](#section_5.2b)
       
       - 5.2c [Temperature related](#section_5.2c)
       
       - 5.2d [Soil moisture related](#section_5.2d)
       
       - 5.2e [Land surface parameters](#section_5.2e)
       
       
   - #### 5.3 [Exploring the correlation between yield and previous year's climate](#section_5.3)

### 6. [Summarize and save the data](#section_6)

---
<a id='section_1'></a>

# 1. Background

Back to [Table of contents](#section_0)

In [ ]:
from IPython.display import Image

<a id='section_1.1'></a>

## 1.1 U.S. Soybean season

From [USDA](https://www.ers.usda.gov/topics/crops/soybeans-oil-crops/oil-crops-sector-at-a-glance/):
> Most U.S. soybeans are planted in May and early June and harvested in late September and October.

### Crop calendars for United States [source](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/us.aspx)

![texte](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/images/us_us_calendar.png)


<a id='section_1.2'></a>

## 1.2 Literature findings

Far from being comprehensive, just list a few useful references to refer to in the future.


- [Hamed et al., 2021, Earth System Dynamics, Impacts of compound hot–dry extremes on US soybean yields](https://esd.copernicus.org/articles/12/1371/2021/)
    - The authors suggest that "The largest negative influence on soybean yields is driven by high temperature and low soil moisture during the summer crop reproductive period."

- [Anderson et al., 2018, Agricultural and Forest Meteorology, Trans-Pacific ENSO teleconnections pose a correlated risk to agriculture](https://www.researchgate.net/publication/326631844_Trans-Pacific_ENSO_teleconnections_pose_a_correlated_risk_to_agriculture)
    - This paper discuss teleconnection impaces of ENSO on maize and soybean growing conditions across the world.
    
- [Zhang et al., 2020, Remote Sensing, Albedo impacts of changing agricultural practices in the United States through Space-Borne Analysis](https://www.researchgate.net/publication/344416659_Albedo_Impacts_of_Changing_Agricultural_Practices_in_the_United_States_through_Space-Borne_Analysis?enrichId=rgreq-c0d44bee512ea316ae24f5fe0ab9c297-XXX&enrichSource=Y292ZXJQYWdlOzM0NDQxNjY1OTtBUzo5NDExMzAwOTA4MDczMzVAMTYwMTM5NDMwNzcwOQ%3D%3D&el=1_x_3&_esc=publicationCoverPdf)
    - This work suggests that "crop growth can result in changes in reflectivity up to a factor of 2 at most wavelength and is unique per crop type in timing and range".  
    
- [Wijewardana et al., 2018, Irrigation Science, Quantifying soil moisture deficit effects on soybean yield and yield component distribution patterns](https://www.researchgate.net/publication/326018870_Quantifying_soil_moisture_deficit_effects_on_soybean_yield_and_yield_component_distribution_patterns)
    - An experimental work to quantify water stress effects on two soybean cultivars with distinct growth traits. 

---
<a id='section_2'></a>
# 2. Import packages and data

Back to [Table of contents](#section_0)

<a id='section_2.1'></a>
## 2.1 Import useful packages 


In [ ]:
import os
import math 
import xarray as xr
import pandas as pd
import numpy  as np
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

import geopandas as gpd
import cmocean

from matplotlib import pyplot as plt
#%config InlineBackend.figure_format = 'retina'
plt.ion()  # To trigger the interactive inline mode

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

from functools import reduce
from netCDF4 import Dataset

import inspect

## Import customized utils

In [ ]:
import utils

---
<a id='section_2.2'></a>

## 2.2 Read the .csv file generated from the data ingestion notebook 

Back to [Table of contents](#section_0)

In [ ]:
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data/')
data = pd.read_csv(data_dir+'GDHY_soybean_NLDAS_1981-2016.csv')
data.head()

---
<a id='section_3'></a>

# 3. EDA of the yield data 

Back to [Table of contents](#section_0)

<a id='section_3.1'></a>

## 3.1 Distribution of annual yield from 1981 to 2016

U.S. soybean annual yield exhibits a bimodal distribution, suggesting that not all U.S. soybean production regions behave in a coherent fashion. My speculation is that it is the different behavior between the Great Plains and other regions (such as Midwest, Southeast, and Northeast) that result in this bimodal distribution pattern. It's likely that grid points over the Great Plains form the left mode, while the rest areas form the right mode. One hint of this speculation is that both the climatology and trend set the Great Plains apart from other areas (see section 3.3 in below), more analysis are needed to further test and understand this hypothesis, and I leave that for future work. 

This bimodal behavior does suggest `a caveat of our approach of viewing all grid points interchangeably.` 

In [ ]:
plt.figure(figsize=(14,6))
sns.distplot(data['yield'])
plt.xlabel('Yield ['+ r'$t$ $ha^{-1}$'+']', fontsize = 20)
plt.ylabel('Density', fontsize = 20)
plt.title('Distribution of U.S. Soybean annual yield', fontsize = 30)

<a id='section_3.2'></a>
---
## 3.2 Boxplot of annual yield from 1981 to 2016

The Interquartile Range (IQR) of Boxplot across all grid points from each year infer the spread across the space. It shows that the spatial spread varies year to year, and many interesting questions can be asked to further understand what affects the spatial spread. 

Changes in the Median value also suggest that the U.S. Soybean experience an overall increasing trend from 1981 to 2016. 

Back to [Table of contents](#section_0)

In [ ]:
sns.set(rc={"figure.figsize":(20, 5)}) #define figure size

ax = sns.boxplot(x='year', y='yield', data=data, whis=10)
ax.set_ylabel('Yield', fontsize= 30)
ax.set_xlabel('Year', fontsize = 30)
ax.tick_params(labelsize=15, rotation=45)
ax.set_title('U.S. Soybean annual yield ['+ r'$t$ $ha^{-1}$'+']', fontsize = 50)

---
<a id='section_3.3'></a>

## 3.3 Climatology, trend, and interannual variability

Back to [Table of contents](#section_0)

### Obtain the yield climatology at each (lat, lon) grid point

In [ ]:
data_yield      = data[['lat', 'lon', 'year', 'yield']]
data_yield_clim = data_yield.groupby(['lat', 'lon']).mean().reset_index()
data_yield_clim.head()

### Obtain the yield trend at each (lat, lon) grid point

In [ ]:
data_yield_trend = data_yield.groupby(['lat', 'lon'])['yield'].apply(lambda x: x.pct_change().mean()).reset_index(name='trend')
data_yield_trend.head()

### Visualize the yield climatology and trend map

Both climatology and trend present a west-east dipole pattern. The contrast between the Great Plains and the rest Soybean production regions stands out. Climatologically, the Great Plains produce less soybean yield compared to other regions. The long-term trend, however, suggests the Great Plains as a hot spot area that experienced soybean yield increases from 1981 to 2016. 

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize = (20, 10))
axes[0] = data_yield_clim.plot(x = 'lon', y = 'lat', kind = 'scatter', c = 'yield', \
                     colormap = 'YlOrRd', vmin = 0, vmax = 5, ax = axes[0])
axes[1] = data_yield_trend.plot(x = 'lon', y = 'lat', kind = 'scatter', c = 'trend', \
                     colormap = 'RdBu_r', vmin=-.1, vmax=.1, ax= axes[1])

axes[0].set_xlabel('Longitude', fontsize = 20)
axes[0].set_ylabel('Latitude', fontsize = 20)
axes[0].set_title('Soybean yield climatology (1981-2016)', fontsize = 20)
axes[0].tick_params(axis='both', which='major', labelsize=20)

axes[1].set_xlabel('Longitude', fontsize = 20)
axes[1].set_ylabel('Latitude', fontsize = 20)
axes[1].set_title('Soybean yield trend (1981-2016)', fontsize = 20)
axes[1].tick_params(axis='both', which='major', labelsize=20)

fig.tight_layout()

### `To do : calculate and visualize the interannual variability of soybean yield.` 

---
<a id='section_4'></a>

# 4. EDA of the climate data 

In this section, we focus on dealiing with missing values in the NLDAS dataset. We will fill the NaNs with the forward interpolation method, and discard variables that have too many missing values. We will focus on `feature selection` in [section 5](#section_5). 

Back to [Table of contents](#section_0)

<a id='section_4.1'></a>

## 4.1 Get familar with climate variables and their standard names

### We will borrow the raw .nc4 file to obtain this information. To ease the narrative, we will use `climate` to denote the `NLDAS` dataset.  

In [ ]:
path_climate         = os.path.join(base_dir, 'downloads/NLDAS/')
test_climate_ds      = xr.open_dataset(path_climate + 'NLDAS_NOAH0125_M.A198101.020.nc.SUB.nc4')
test_climate_df      = test_climate_ds.to_dataframe()

# .reset_index() helps to assign lat and lon for each entry
test_climate_df      = test_climate_df.reset_index()

# Read in bnds 0 only, and sort values by the ascending order of lat and lon
test_climate_sorted  = test_climate_df[test_climate_df['bnds'] == 0].sort_values(by = ['lat', 'lon'])
climate_column_names = list(test_climate_sorted)
climate_column_names

### Create a list named `climate_variable_list` to contain all the climate variable column names

In [ ]:
climate_variable_list = climate_column_names[5:len(climate_column_names)]
climate_variable_list

### Create a dictionary `climate_variable_dict` to include paired climate variable column name, standard name, and units

The standard name helps us to understand the physical meaning of each variable. 

In [ ]:
climate_variable_name_units = []
for index in range(0,len(climate_variable_list)):
    name      = test_climate_ds[climate_variable_list[index]].standard_name
    units     = test_climate_ds[climate_variable_list[index]].units
    name_units= name + ' ('+units+')'
    climate_variable_name_units.append(name_units)

climate_variable_dict = dict(zip(climate_variable_list, climate_variable_name_units))
climate_variable_dict

---
<a id='section_4.2'></a>


## 4.2 Check the missing values

Back to [Table of contents](#section_0)

### First to print out the `data.info` to remind of ourselves about the dataframe content

### We have 53384 records. 

In [ ]:
data.info

### We have 641 columns, with the climate related columns start at index 5. 

In [ ]:
data_columns        = data.columns
data_columns

### We create `data_climate_columns` by indexing `data_columns` from the 5th to the last index.  

### Thus, `data_climate_columns` contains all climate related column names in the data dataframe.

In [ ]:
data_climate_columns = data_columns[5:641]
data_climate_columns

### We now check the counts of NaN values for each climate column. 

The printed outputs suggest that most variables contain 285 missing values across the dataframe. It's likely that these missing values locate at the same areas for different variables. Streamflow has much more missing points (the count is 3170).

In [ ]:
for index in range(0,len(data_climate_columns)):
    column = data_climate_columns[index]
    if data[column].isnull().values.any():
        print('NaN exists ('+str(data[column].isna().sum())+') : '+ column)

---
## Where do the missing values locate? 

### Let's do a quick check. Here, we use `SWdown_Jan` and `Streamflow_Jan` as example, to get a sense of where the missing value locate, and why the Streamflow has so many missing values. 

As shown below, `SWdown_Jan` and `Rainf_Jan` share the same missing value locations, and there are only a few missing value across the domain for each month. We did not check other variables that have the same total amount of NaN values, but we expect they all share the same locations for missing values. It won't matter even if not, because our goal is to make sure there are not many values missing and it's easy for us to fill the missing value with interpolation. 

However, `Streamflow_Jan` is a different creature. A lot of streamflow values are missing in the Northeast U.S.

In [ ]:
def plot_NaN(var):
    """
    Visualize where do the Nan values in the climate dataframe locate.
    
    Parameters
    ----------
    var: string
        variable name of interest
        
    Returns
    ----------
    plot 
    
    """
    
    data_column   = data[var]
    data_NaN      = data[['lat', 'lon', var]]
    
    data_NaN[var] = 0
    
    for index in range(0, len(data)):
        if math.isnan(data_column[index]):
            data_NaN[var][index] = 10
            
    #sns.set(rc={"figure.figsize":(8, 5)}) #width=8, height=4
    fig, ax = plt.subplots(figsize=(8,5))
    data_NaN.plot(x = 'lon', y = 'lat', kind = 'scatter', c = var, colormap = 'Reds', ax = ax)
    ax.set_xlabel('Longitude', fontsize = 20)
    ax.set_ylabel('Latitude', fontsize = 20)
    ax.set_title(var + '(missing value highlighted)', fontsize = 20)

In [ ]:
plot_NaN('SWdown_Jan')

In [ ]:
plot_NaN('Rainf_Jan')

In [ ]:
plot_NaN('Streamflow_Jan')

---
<a id='section_4.3'></a>


## 4.3 Fill the missing values


### We make a copy of `data` and then fill the NaN values using the interpolation method

<div class="alert alert-block alert-danger">
Note that we use the deep copy to make sure any future changes in the new dataframe won't be reflected in the original copy.
</div>

In [ ]:
data_filled        = data.copy(deep = True)
for index in range(0,len(data_columns)):
    column = data_columns[index]
    if data_filled[column].isnull().values.any():
        data_filled[column].interpolate(method='linear', direction='forward', inplace=True)

### Print out those variables that still contain NaN after the interpolation

It turns out that Streamflow still contains NaN after the interpolation method, likely because there are a bunch of NaN points in the Northeastern U.S. whose nearest neighbors are also NaNs.

In [ ]:
for index in range(0,len(data_columns)):
    column = data_columns[index]
    if data_filled[column].isnull().values.any():
        print('NaN exists ('+str(data_filled[column].isna().sum())+') : '+ column)

---
<a id='section_5'></a>

# 5. Feature selection 
    
Back to [Table of contents](#section_0)

Now, let's decide which features to keep as predictors. Recall that we have multiple variables from the NLDAS data, and we will likely won't need that many for our modeling. 

### Our mental model of feature selection for the soybean yield prediction works like this:

    1. Given that the typical U.S. soybean season spans May through October (May-June: Plant; Jul-August: Mid-season; September-October: Harvest), November and December climate information from the current year won't tell us much about how the soybean production performs in the current year. We thus drop all the current November and December variables. 
    
    2. We will detect the important features based on the correlation heatmap between annual yield and climate features; we will keep variables that show strong correlation with yield, and discard those showing weak correlations. We will do some sanity checks while working with the correlation heatmaps:
        2.1 Some climate variables are strong correalted with each other, therefore for those features that exhibit strong correlation with yield, we want to only include the most relevant one and remove the redundant ones. 
        
        2.2 The correlation heatmap approach is based on the assumption of a linear correlation between yield and climate data. However, yield may have nonlinear relationship with some variables, such as temperature, precipitation and snow related variables. Thus, it's worth to show other visualization to decide whether to keep some variables or not. 
    
    3. Some variables directly reflect the yield data. For example, leaf area index (LAI) has strong correlation with the yield data, but does not necessarily provide predictive power. That being said, these vegetation and land cover related features from earlier seasons may have a predictive power.  

### Workflow of this section:

   5.1 [Drop current Nov-Dec columns and streamflow data](#section_5.1)
   
   5.2 [Exploring the correlation between yield and current monthly climate](#section_5.2)
   
   5.3 [Exploring the correlation between yield and previous year climate](#section_5.3)


---
## We do feature selection in each subsections of section 5. We create two empty lists to track all columns to drop / to keep

- ### `drop_columns` - all feature columns that will be dropped out of `df_target_feature`

- ### `keep_columns` - all feature columns that stay in or add to `df_target_feature`

In [ ]:
drop_columns = []
keep_columns = []

---
<a id='section_5.1'></a>

## 5.1 Drop current Nov-Dec columns and streamflow data

Back to [Table of contents](#section_0)

We create a new dataframe named `df_target_feature` to store the yield data and all final features that we want to include in our model. We first drop current Nov and Dec columns and streamflow data and save as `df_target_feature`, and we will continue updating `df_target_feature` by dropping other irrelevant features and including engineered variables. 

We will save this `df_target_feature` at the end of the notebook, and will feed it to our model for predictions. 

### Copy `data_filled` to `df_target_feature`

<div class="alert alert-block alert-danger">
Note that we use the deep copy to make sure any future changes in the new dataframe won't be reflected in the original copy.
</div>

In [ ]:
df_target_feature = data_filled.copy(deep = True)

### Columns to be dropped from 5.1: Nov, Dec, and Streamflow

In [ ]:
drop_Nov_Dec_streamflow = [col for col in data_filled.columns if col.endswith('_Nov') or \
                                                                 col.endswith('_Dec') or \
                                                                 col.startswith('Streamflow_')]

In [ ]:
drop_columns.append(drop_Nov_Dec_streamflow)

### Drop the columns (`set inplace = True to directly update df_target_feature`)

In [ ]:
df_target_feature.drop(drop_Nov_Dec_streamflow, axis = 1, inplace=True)
df_target_feature

<a id='save_raw_feature'></a>

### Save this `df_target_feature` as a raw feature dataframe 

In [ ]:
df_target_feature.to_csv(data_dir+'US_SoybeanYield_ClimateFeature_1981-2016_v0.csv')

---
<a id='section_5.2'></a>

## 5.2 Exploring the correlation between yield and current monthly climate

Back to [Table of contents](#section_0)

### We group the variables that have closer physical process/meanings/effects together, and will explore each of the following five subsection one by one.

<div class="alert alert-block alert-info">

### [5.2a Suface energy flux variables:](#section_5.2a)

- 'SWdown': 'Shortwave radiation flux downwards (surface) (W m-2)'
- 'LWdown': 'Longwave radiation flux downwards (surface) (W m-2)'
- 'SWnet': 'Net shortwave radiation flux (surface) (W m-2)'
- 'LWnet': 'Net longwave radiation flux (surface) (W m-2)'
- 'Qle': 'Latent heat flux (W m-2)'
- 'Qh': 'Sensible heat flux (W m-2)'
- 'Qg': 'Ground heat flux (W m-2)'
- 'Qf': 'Snow phase-change heat flux (W m-2)'
</div>

<div class="alert alert-block alert-info">
    
### [5.2b Surface water flux / storage variables:](#section_5.2b)

- 'Snowf': 'Frozen precipitation (snowfall) (kg m-2)'
- 'Rainf': 'Liquid precipitation (rainfall) (kg m-2)'
- 'Evap': 'Total evapotranspiration (kg m-2)'
- 'Qs': 'Surface runoff (non-infiltrating) (kg m-2)'
- 'Qsb': 'Subsurface runoff (baseflow) (kg m-2)'
- 'Qsm': 'Snowmelt (kg m-2)'
- 'SWE': 'Snow Water Equivalent (kg m-2)'
- 'SnowDepth': 'Snow depth (m)'
- 'SnowFrac': 'Snow cover (fraction)'
- 'PotEvap': 'Potential evapotranspiration (W m-2)'
- 'ECanop': 'Canopy water evaporation (W m-2)'
- 'TVeg': 'Transpiration (W m-2)'
- 'ESoil': 'Direct evaporation from bare soil (W m-2)'
- 'SubSnow': 'Sublimation (evaporation from snow) (W m-2)'
- 'CanopInt': 'Plant canopy surface water (kg m-2)'
</div>

<div class="alert alert-block alert-info">

### [5.2c Temperature related variables:](#section_5.2c)
    
- 'AvgSurfT': 'Average surface skin temperature (K)'
- 'Albedo': 'Surface albedo (%)'
- 'SoilT_0_10cm': 'Soil temperature (0-10cm) (K)'
- 'SoilT_10_40cm': 'Soil temperature (10-40cm) (K)'
- 'SoilT_40_100cm': 'Soil temperature (40-100cm) (K)'
- 'SoilT_100_200cm': 'Soil temperature (100-200cm) (K)'
</div>

<div class="alert alert-block alert-info">

### [5.2d Soil moisture related:](#section_5.2d) 
   
- 'SoilM_0_10cm': 'Soil moisture content (0-10cm) (kg m-2)'
- 'SoilM_10_40cm': 'Soil moisture content (10-40cm) (kg m-2)'
- 'SoilM_40_100cm': 'Soil moisture content (40-100cm) (kg m-2)'
- 'SoilM_100_200cm': 'Soil moisture content (100-200cm) (kg m-2)'
- 'SoilM_0_100cm': 'Soil moisture content (0-100cm) (kg m-2)'
- 'SoilM_0_200cm': 'Soil moisture content (0-200cm) (kg m-2)'
- 'RootMoist': 'Root zone soil moisture (kg m-2)'
  
- 'SMLiq_0_10cm': 'Liquid soil moisture content (0-10cm) (kg m-2)'
- 'SMLiq_10_40cm': 'Liquid soil moisture content (10-40cm) (kg m-2)'
- 'SMLiq_40_100cm': 'Liquid soil moisture content (40-100cm) (kg m-2)'
- 'SMLiq_100_200cm': 'Liquid soil moisture content (100-200cm) (kg m-2)'

- 'SMAvail_0_100cm': 'Soil moisture availability (0-100cm) (%)'
- 'SMAvail_0_200cm': 'Soil moisture availability (0-200cm) (%)'
</div>

<div class="alert alert-block alert-info">

### [5.2e Vegetation and land surface parameters:](#section_5.2e)    

These parameters are closely related to surface vegetation cover, so likely a direct reflection of the crop yield in the current year. 

- 'LAI': 'Leaf Area Index (unitless)'
- 'GVEG': 'Green vegetation (fraction)'
- 'ACond': 'Aerodynamic conductance (m s-1)'
- 'CCond': 'Canopy conductance (m s-1)'
- 'RCS': 'Solar parameter in canopy conductance (fraction)'
- 'RCT': 'Temperature parameter in canopy conductance (fraction)'
- 'RCQ': 'Humidity parameter in canopy conductance (fraction)'
- 'RCSOL': 'Soil moisture parameter in canopy conductance (fraction)'
- 'RSmin': 'Minimal stomatal resistance (s m-1)'
- 'RSMacr': 'Relative soil moist. availability control factor (0-1)'

</div>

---
### Use our customized functions for visualization  

#### `Function: plot_corr_yield_current_climate`

In [ ]:
print(inspect.getdoc(utils.plot_corr_yield_current_climate))

#### `Function: plot_corr_climate`

In [ ]:
print(inspect.getdoc(utils.plot_corr_climate))

#### `Function: plot_corr_climate_selected_months`

In [ ]:
print(inspect.getdoc(utils.plot_corr_climate_selected_months))

#### `Function: panel_scatter`

In [ ]:
print(inspect.getdoc(utils.panel_scatter))

---


<a id='section_5.2a'></a>

<div class="alert alert-block alert-info">

### 5.2a Suface energy flux variables (jump to the [5.2a summary](#section_5.2a_summary))
    
- 'SWdown': 'Shortwave radiation flux downwards (surface) (W m-2)'
- 'LWdown': 'Longwave radiation flux downwards (surface) (W m-2)'
- 'SWnet': 'Net shortwave radiation flux (surface) (W m-2)'
- 'LWnet': 'Net longwave radiation flux (surface) (W m-2)'
- 'Qle': 'Latent heat flux (W m-2)'
- 'Qh': 'Sensible heat flux (W m-2)'
- 'Qg': 'Ground heat flux (W m-2)'
- 'Qf': 'Snow phase-change heat flux (W m-2)'

</div>

Back to [Table of contents](#section_0)

### In section 5.2a, we first explor the radiative fluxes: `SWdown`, `LWdown`, `SWnet`, `LWnet`


#### `Figure: Yield versus SWdown, LWdown, SWnet, and LWnet`

- SWdown and SWnet show almost identical relationship with yield, they both show negative correlation with yield throughout the whole year. The negative correlation between shortwave radiation and yield is particularly strong in June and July.

- Both LWdown and LWnet show positive correlation with yield, but the magnitude of correlation coefficients differs a lot. The correlation between LWdown and yield is very weak, while the LWnet exhibits strong positive correlation with yield, especially during the soybean season. 

In [ ]:
utils.plot_corr_yield_current_climate(df_target_feature, \
                                      ['SWdown', 'LWdown', 'SWnet', 'LWnet'], \
                                      climate_variable_dict)

### Feature engineering: `JJA_SWdown` and `JJA_LWnet`

- Compared to SWnet and LWdown, SWdown and LWnet appear to have a stornger correlation with yield

- Not all months show a strong correlation, so let's just focus on JJA (June to August), which show stronger correlation with yield. 

In [ ]:
df_target_feature['JJA_SWdown'] = df_target_feature[['SWdown_Jun', 'SWdown_Jul', 'SWdown_Aug']].mean(axis=1)
df_target_feature['JJA_LWnet']  = df_target_feature[['LWnet_Jun', 'LWnet_Jul', 'LWnet_Aug']].mean(axis=1)

climate_variable_dict.update({'JJA_SWdown': 'June to August averaged surface incoming solar radiation (W/m2)'})
climate_variable_dict.update({'JJA_LWnet': 'June to August averaged surface net longwave radiation (W/m2)'})

### Double-check the aggregated SWdown and LWnet correlation with yield, compared to the individual summer months

- Both JJA_SWdown and JJA_LWnet perform better than the individual summer month's SWdown and LWnet

- JJA_SWdown and JJA_LWnet are negatively correlated, but not too much, the correlation coefficient is ~ - 0.56

- JJA_SWdown is highly correlated with SWdown_Jun, SWdown_Jul, and SWdown_Aug; JJA_LWnet is highly correlated with LWnet_Jun, LWnet_JUl, and LWnet_Aug, again suggesting that it's reasonable to keep JJA_SWdown and JJA_LWnet only. 

In [ ]:
df_target_feature[['yield', 'JJA_SWdown', 'JJA_LWnet', \
                   'SWdown_Jun', 'SWdown_Jul', 'SWdown_Aug', 'LWnet_Jun', 'LWnet_Jul', 'LWnet_Aug']].corr()

---

### We now explore other surface energy fluxes: `Qle`, `Qh`, `Qg`, `Qf`



### Feature engineering: `evaporative fraction (EF)`

- Evaporative fraction, defined as the latent heat flux divided by the net available surface energy (e.g., SWnet - G), might be a better predictor to capture the impacts of surface turbulent heat flux partitioning on soybean yield.

In [ ]:
month_suffix = ['_Jan', '_Feb', '_Mar', '_Apr', '_May', '_Jun', '_Jul', '_Aug', '_Sep', '_Oct']

for month in month_suffix:
    df_target_feature['EF'+month] = df_target_feature['Qle'+month] / \
    (df_target_feature['SWnet'+month] - df_target_feature['Qg'+month])

climate_variable_dict.update({'EF': 'Evaporative fraction (unitless)'})

#### `Figure: Yield versus Qle, Qh, EF, Qg, Qf` 

- Discard Qf and Qh due to very weak correlation 

- Both Qle and EF has modest correlation with yield, it's likely that Qle and EF are also highly correlatd, so we will choose one from Qle and EF. 


In [ ]:
utils.plot_corr_yield_current_climate(df_target_feature, \
                                     ['Qle', 'Qh', 'EF', 'Qg', 'Qf'], \
                                     climate_variable_dict)

#### `Figure: correlation between SWdown & EF, LWnet & EF, and Qle & EF`

- Qle and EF are highly correlated, so we only keep one. Given that EF has stronger positive correlation with yield, we keep EF and discard Qle. 

- SWdown and EF do not have concerning strong correlations, but LWnet is highly correlated with EF in Septermber and October. 

- We need to decide whether we want to do some aggregation of EF. 

- I do not fully understand how Qg affect yield. Qg is often viewed as a residual of other surface energy fluxes. To ease the final interpretation, I decide to discard this feature for now, despite it may provide some predictive power. 


In [ ]:
utils.plot_corr_climate(df_target_feature, ['SWdown', 'EF'], ['LWnet', 'EF'], ['Qle', 'EF'], ['LWnet', 'Qg'])

### Decide how to include EF: selected months or aggregation?

In [ ]:
df_target_feature['JFMA_EF'] = df_target_feature[['EF_Jan', 'EF_Feb', 'EF_Mar', 'EF_Apr']].mean(axis=1)
df_target_feature['MJ_EF']   = df_target_feature[['EF_May', 'EF_Jun']].mean(axis=1)
df_target_feature['JJA_EF']  = df_target_feature[['EF_Jun', 'EF_Jul', 'EF_Aug']].mean(axis=1)
df_target_feature['JA_EF']   = df_target_feature[['EF_Jul', 'EF_Aug']].mean(axis=1)
df_target_feature['ASO_EF']  = df_target_feature[['EF_Aug', 'EF_Sep', 'EF_Oct']].mean(axis=1)
df_target_feature['SO_EF']   = df_target_feature[['EF_Sep', 'EF_Oct']].mean(axis=1)

df_target_feature[['yield', 'JFMA_EF', 'MJ_EF', 'JA_EF', 'JJA_EF','ASO_EF', 'SO_EF']].corr()

#### `Figure: scatter between yield and different EF options`

The above .corr table and the below scatter plots suggest:

- Both Jan-Apr and Sep-Oct averaged EF show somewhat modest positive correlation with yield, not too strong, close to 0.5. JFMA_EF and SO_EF are positively correlated, around 0.6, so not too concerning. 

- MJ_EF is not useful.

- JA_EF and JJA_EF are very similar, given that JA_EF has a higher correlation with SO_EF, we will keep JJA_EF. 

In [ ]:
utils.panel_scatter(df_target_feature, \
             ['JFMA_EF', 'yield'], \
             ['MJ_EF', 'yield'], \
             ['JA_EF', 'yield'], \
             ['JJA_EF', 'yield'], \
             ['SO_EF', 'yield'])

<a id='section_5.2a_summary'></a>

<div class="alert alert-block alert-success">

### 5.2a Suface energy flux variables Summary (back to [the beginning of 5.2a](#section_5.2a))
    
<b>Discard all</b>
    
   - 'LWnet': 'Net longwave radiation flux (surface) (W m-2)'
    
   - 'SWdown': 'Shortwave radiation flux downwards (surface) (W m-2)'
    
   - 'LWdown': 'Longwave radiation flux downwards (surface) (W m-2)'
    
   - 'SWnet': 'Net shortwave radiation flux (surface) (W m-2)' 
    
   - 'Qle': 'Latent heat flux (W m-2)' 
    
   - 'Qh': 'Sensible heat flux (W m-2)' 
    
   - 'Qf': 'Snow phase-change heat flux (W m-2)' 
    
   - 'Qg': 'Ground heat flux (W m-2)'
    

<b>Discard engineered and aggregated features</b>

   - 'MJ_EF'
    
   - 'JA_EF'
    
   - 'ASO_EF'
    
<b>Aggregated and Engineered features to keep</b>
    
   - 'JJA_LWnet'
    
   - 'JJA_SWdown'
    
   - 'JFMA_EF'
    
   - 'JJA_EF'
    
   - 'SO_EF'
    
</div>

### Finalize the feature selection from 5.2a

   - #### `drop_energy_fluxes` - columns to drop
   - #### `keep_energy_fluxes` - columns to keep

In [ ]:
drop_energy_fluxes = [col for col in df_target_feature.columns if col.startswith('LWnet_') or \
                                                                  col.startswith('SWdown_') or \
                                                                  col.startswith('LWdown_') or \
                                                                  col.startswith('SWnet_') or \
                                                                  col.startswith('Qle_') or \
                                                                  col.startswith('Qh_') or \
                                                                  col.startswith('Qf_') or \
                                                                  col.startswith('Qg_') or \
                                                                  col.startswith('EF_')] \
                    + ['MJ_EF', 'JA_EF', 'ASO_EF']

                   

keep_energy_fluxes = ['JJA_LWnet', 'JJA_SWdown', 'JFMA_EF', 'JJA_EF', 'SO_EF']

df_target_feature.drop(drop_energy_fluxes, axis=1, inplace=True)

### Sanity check the `drop_energy_fluxes` columns are gone, and the `keep_energy_fluxes` remain.

In [ ]:
for col in drop_energy_fluxes:
    if col in df_target_feature.columns:
        print(col +' should be removed, double check your procedure.')

for col in keep_energy_fluxes:
    if not col in df_target_feature.columns:
        print(col +' should be retained, double check your procedure.')

### Update `drop_columns` and `keep_columns`

In [ ]:
drop_columns.append(drop_energy_fluxes)
keep_columns.append(keep_energy_fluxes)

---
<a id='section_5.2b'></a>
<div class="alert alert-block alert-info">

### 5.2b Surface water flux / storage variables (jump to the [5.2b summary](#section_5.2b_summary))
    
#### Rain and Snow related
- 'Rainf': 'Liquid precipitation (rainfall) (kg m-2)'
- 'Snowf': 'Frozen precipitation (snowfall) (kg m-2)'
- 'Qsm': 'Snowmelt (kg m-2)'
- 'SWE': 'Snow Water Equivalent (kg m-2)'
- 'SnowDepth': 'Snow depth (m)'
- 'SnowFrac': 'Snow cover (fraction)'
- 'SubSnow': 'Sublimation (evaporation from snow) (W m-2)'

#### Evaporation related
- 'Evap': 'Total evapotranspiration (kg m-2)'
- 'PotEvap': 'Potential evapotranspiration (W m-2)'
- 'ECanop': 'Canopy water evaporation (W m-2)'
- 'TVeg': 'Transpiration (W m-2)'
- 'ESoil': 'Direct evaporation from bare soil (W m-2)'
- 'CanopInt': 'Plant canopy surface water (kg m-2)'

#### Runoff related
- 'Qs': 'Surface runoff (non-infiltrating) (kg m-2)'
- 'Qsb': 'Subsurface runoff (baseflow) (kg m-2)'
    
</div>

Back to [Table of contents](#section_0)

---
### Selecting the snow related variable

We expect that only snow from earlier season matter, so we first double-check the correlations across the snow variables in January to April to determine whether we can remove redundant snow features if they are highly correlated. 

`As shown in below, Snowf is highly correlated with SWE, Qsm, SnowDepth, SnowFrac, SubSnow. We can directly discard those redundant features and keep Snowf only.`

In [ ]:
spring_months     = ['_Jan', '_Feb', '_Mar', '_Apr']
spring_Snowf      = [sub.replace('_', 'Snowf_') for sub in spring_months]
spring_SWE        = [sub.replace('_', 'SWE_') for sub in spring_months]
spring_Qsm        = [sub.replace('_', 'Qsm_') for sub in spring_months]
spring_SnowDepth  = [sub.replace('_', 'SnowDepth_') for sub in spring_months]
spring_SnowFrac   = [sub.replace('_', 'SnowFrac_') for sub in spring_months]
spring_SubSnow    = [sub.replace('_', 'SubSnow_') for sub in spring_months]

In [ ]:
utils.plot_corr_climate_selected_months(df_target_feature, \
                                       [spring_Snowf, spring_SWE, 'Snowf vs  SWE'],\
                                       [spring_Snowf, spring_Qsm, 'Snowf vs Qsm'], \
                                       [spring_Snowf, spring_SnowDepth, 'Snowf vs SnowDepth'],\
                                       [spring_Snowf, spring_SnowFrac, 'Snowf vs SnowFrac'], \
                                       [spring_Snowf, spring_SubSnow, 'Snowf vs SubSnow'])

### Selecting the evaporation related variable

We want to check whether the evaporation variables are redundant compared to features we already decided to retain. For example, we expect a high correlation between `SWdown` and `PotEvap` because the potential evapotranspiration reflects the net available energy at surface. Further, the `Evap` variable is equivalent to `Qle` so we expect it has high correlation with `EF`, as `Qle` does. Finally, we expect some correlation between `Evap` and other evaporative process such as `ECanop` and `TVeg`, both of which contribute to `Evap`. 

`As shown in below, indeed the above mentioned variables are highly correlated with each other. Thus, we will not include PotEvap, Evap, ECanop, and TVeg.`

`At this moment, we are unsure about ESoil yet; ESoil does not have considerable correlation with other evaporation processes, we will keep ESoil and will check its connection with soil moisture later when we explore soil moisture variables. We also keep CanopInt for now as it does not seem to relate to Evap in a direct way.`

In [ ]:
utils.plot_corr_climate(data_filled, \
                       ['SWdown', 'PotEvap'], \
                       ['Evap', 'EF'], \
                       ['Evap', 'ECanop'], \
                       ['Evap', 'TVeg'], \
                       ['Evap', 'ESoil'], \
                       ['Evap', 'CanopInt'])   

### Selecting runoff variables 

We want to check whether the `Qs` and `Qsb` are somehow related, or whether these surface/subsurface runoff are connected with other hydrology controlling factors or processes. 

`It seems that Qs and Qsb are only weakly correlated and neither of the runoff variables present concerning strong correlations with evaporation, soil evaporation, and canopy surface water. So we decide to keep both Qs and Qsb for now.`

In [ ]:
utils.plot_corr_climate(df_target_feature, \
                       ['Qs', 'Qsb'], \
                       ['Qs', 'ESoil'], \
                       ['Qs', 'CanopInt'], \
                       ['Qs', 'EF'], \
                       ['Qs', 'SWdown'])

---
### Based on our above exploration, we narrow our feature selection to following variables:

   - 'Rainf': 'Liquid precipitation (rainfall) (kg m-2)'
   
   - 'Snowf': 'Frozen precipitation (snowfall) (kg m-2)'
   
   - 'ESoil': 'Direct evaporation from bare soil (W m-2)'
    
   - 'CanopInt': 'Plant canopy surface water (kg m-2)'
    
   - 'Qs': 'Surface runoff (non-infiltrating) (kg m-2)'
    
   - 'Qsb': 'Subsurface runoff (baseflow) (kg m-2)'

### We expect `Rainf` to be an important feature, and heavier rainfall may lead to more canopy surface water and surface runoff. To explore whether we can further narrow down our feature selection, we explore the correlation between `Rainf` and the rest hydrology variables. 

`Both Qs and Qsb are strongly correlated with Rainf, with the correlation coefficient exceeding 0.5 in most months. Canopy surface water also has strong correlation with Rainf in April throughout October. Rainf does not correlate with Snowf, and only have strong correlation with ESoil during the beginning months of the year.`

`So, let's ignore CanopInt, Qs, Qsb, and Jan to March of ESoil. Now we are left with Rainf, Snowf, ESoil, and we can further narrow down the search with these three variables.`

In [ ]:
utils.plot_corr_climate(df_target_feature, \
                       ['Rainf', 'ESoil'], \
                       ['Rainf', 'CanopInt'], \
                       ['Rainf', 'Qs'], \
                       ['Rainf', 'Qsb'], \
                       ['Rainf', 'Snowf'])

---
### Narrow down to `Rainf`, `Snowf`, and `ESoil`  

We now explore the correlation between `yield` and `Rainf`, `Snowf`, and `ESoil`, and decide whether we want to keep all months, or if we just keep selected months from each variable.


In [ ]:
utils.plot_corr_yield_current_climate(df_target_feature, \
                                      ['Rainf', 'Snowf', 'ESoil'], \
                                      climate_variable_dict)

### Can we do some aggregation for Rainf, Snowf, and ESoil?

- Keep JFMA_Rainf, MJJ_Rainf, JF_Snowf, and JJA_ESoil 

- JFMA_Rainf has a modest positive correlation with yield

- MJJ_Rainf appear to correlate with yield in a nonlinear fashion 

- JF_Snowf & Yield: a weak positive correlation 

- JJA_ESoil & Yield: negatively correlated


In [ ]:
df_target_feature['JFMA_Rainf'] = df_target_feature[['Rainf_Jan', 'Rainf_Feb', 'Rainf_Mar', 'Rainf_Apr']].mean(axis=1)
df_target_feature['MJJ_Rainf']  = df_target_feature[['Rainf_May', 'Rainf_Jun', 'Rainf_Jul']].mean(axis=1)
df_target_feature['JF_Snowf'] = df_target_feature[['Snowf_Jan', 'Snowf_Feb']].mean(axis=1)
df_target_feature['JJA_ESoil'] = df_target_feature[['ESoil_Jun', 'ESoil_Jul', 'ESoil_Aug']].mean(axis=1)

df_target_feature[['yield', 'JJA_SWdown', 'JJA_LWnet', 'JFMA_EF', 'JJA_EF', 'JFMA_Rainf', 'MJJ_Rainf', \
                   'JF_Snowf', 'JJA_ESoil']].corr()

### Scatter plots between yield and each of the aggregated variables

In [ ]:
plt.plot(df_target_feature['JFMA_Rainf'], df_target_feature['yield'], 'o', alpha = 0.5)
plt.xlim(0,500)

In [ ]:
plt.plot(df_target_feature['MJJ_Rainf'], df_target_feature['yield'], 'o', alpha = 0.5)
plt.xlim(0,500)

In [ ]:
plt.plot(df_target_feature['JF_Snowf'], df_target_feature['yield'], 'o', alpha = 0.5)


In [ ]:
plt.plot(df_target_feature['JJA_ESoil'], df_target_feature['yield'], 'o', alpha = 0.5)

<a id='section_5.2b_summary'></a>

<div class="alert alert-block alert-success">
    
### 5.2b Surface water flux / storage variables Summary (back to [the beginning of 5.2b](#section_5.2b))
    
<b>Discard all</b>

   - 'Qsm': 'Snowmelt (kg m-2)'
    
   - 'SWE': 'Snow Water Equivalent (kg m-2)'

   - 'SnowDepth': 'Snow depth (m)'

   - 'SnowFrac': 'Snow cover (fraction)'

   - 'SubSnow': 'Sublimation (evaporation from snow) (W m-2)'
   
   - 'Evap': 'Total evapotranspiration (kg m-2)'
    
   - 'PotEvap': 'Potential evapotranspiration (W m-2)'

   - 'ECanop': 'Canopy water evaporation (W m-2)'

   - 'TVeg': 'Transpiration (W m-2)'
    
   - 'CanopInt': 'Plant canopy surface water (kg m-2)'
    
   - 'Qs': 'Surface runoff (non-infiltrating) (kg m-2)'
    
   - 'Qsb': 'Subsurface runoff (baseflow) (kg m-2)'
    
   - 'Snowf': 'Frozen precipitation (snowfall) (kg m-2)'
    
   - 'ESoil': 'Direct evaporation from bare soil (W m-2)'
   
   - 'Rainf': 'Liquid precipitation (rainfall) (kg m-2)'
       
<b>Aggregated features to keep</b>
    
   - 'JFMA_Rainf', 
    
   - 'MJJ_Rainf'
    
   - 'JF_Snowf'
    
   - 'JJA_ESoil'
    
</div>

### Finalize feature selection 

   - #### `drop_hydrology` - columns to drop
   - #### `keep_hydrology` - columns to keep

In [ ]:
drop_hydrology = [col for col in df_target_feature.columns if col.startswith('Qsm_') or \
                                                              col.startswith('SWE_') or \
                                                              col.startswith('SnowDepth_') or \
                                                              col.startswith('SnowFrac_') or \
                                                              col.startswith('SubSnow_') or \
                                                              col.startswith('Evap_') or \
                                                              col.startswith('PotEvap_') or \
                                                              col.startswith('ECanop_') or \
                                                              col.startswith('TVeg_') or \
                                                              col.startswith('CanopInt_') or \
                                                              col.startswith('Qs_') or \
                                                              col.startswith('Qsb_') or \
                                                              col.startswith('Snowf_') or \
                                                              col.startswith('ESoil_') or \
                                                              col.startswith('Rainf_')]

keep_hydrology = ['JFMA_Rainf', 'MJJ_Rainf', 'JF_Snowf', 'JJA_ESoil']

df_target_feature.drop(drop_hydrology, axis=1, inplace=True)

### Sanity check that `drop_hydrology` are gone and `keep_hydrology` remain

In [ ]:
for col in drop_hydrology:
    if col in df_target_feature.columns:
        print(col +' should be removed, double check your procedure.')

for col in keep_hydrology:
    if not col in df_target_feature.columns:
        print(col +' should be retained, double check your procedure.')

### Update `drop_columns` and `keep_columns`

In [ ]:
drop_columns.append(drop_hydrology)
keep_columns.append(keep_hydrology)

---
<a id='section_5.2c'></a>
<div class="alert alert-block alert-info">

### 5.2c Temperature related variables (jump to the [5.2c summary](#section_5.2c_summary))

surface temperature
- 'AvgSurfT': 'Average surface skin temperature (K)'
- 'Albedo': 'Surface albedo (%)'

soil temperature
- 'SoilT_0_10cm': 'Soil temperature (0-10cm) (K)'
- 'SoilT_10_40cm': 'Soil temperature (10-40cm) (K)'
- 'SoilT_40_100cm': 'Soil temperature (40-100cm) (K)'
- 'SoilT_100_200cm': 'Soil temperature (100-200cm) (K)'
    
</div>

Back to [Table of contents](#section_0)

### Check temperature feature redundancy

`AvgSurfT` is highly correlated with the soil temperature across all levels. Safe to discard all soil temperature variables. 

In [ ]:
utils.plot_corr_climate(df_target_feature, \
                       ['AvgSurfT', 'SoilT_0_10cm'], \
                       ['AvgSurfT', 'SoilT_10_40cm'], \
                       ['AvgSurfT', 'SoilT_40_100cm'], \
                       ['AvgSurfT', 'SoilT_100_200cm'])

### Check the redundancy between `AvgSurfT` and features that we decided to retain from 5.2a and 5.2b. Particularly, the connection between `AvgSurfT` and `SWdown`, `EF`, and `ESoil`

`AvgSurfT` is highly correlated with `SWdown`, and it has considerable correlation with `EF` and `ESoil`, which makes sense. Therefore no need to keep `AvgSurfT` either. 

In [ ]:
utils.plot_corr_climate(data_filled, \
                       ['AvgSurfT', 'SWdown'], \
                       ['AvgSurfT', 'EF'], \
                       ['AvgSurfT', 'ESoil'])

### What about `Albedo`?

It suggests that `yield` has a constant negative correlation with `Albedo` throughout its planting, mid-season, and harvest season, suggesting a decrease in albedo is beneficial for soybean annual yield. There are many factors that could affect albedo and we do not dig deeper on that front. Previous studies show that albedo over soybean cropland increases during the soybean season, see [this paper](https://digitalcommons.unl.edu/cgi/viewcontent.cgi?article=1175&context=natrespapers), for example. Therefore, the negative correlation shown here might suggest potential causal linkage of albedo effects on soybean yield, rather just a simple reflection of the yield performance itself. 

In [ ]:
utils.plot_corr_yield_current_climate(df_target_feature, \
                                     ['Albedo'], \
                                     climate_variable_dict)

### Aggregation of albedo

Note that thought May to October Albedo show negative correlation with yield exceeding -0.5, the albedo is highly correlated across months, so we do some aggregation. 

In [ ]:
df_target_feature['MJJA_Albedo'] = df_target_feature[['Albedo_May', 'Albedo_Jun', \
                                                      'Albedo_Jul', 'Albedo_Aug']].mean(axis=1)

df_target_feature[['yield', 'JJA_SWdown', 'JJA_LWnet', 'MJJA_Albedo']].corr()

<a id='section_5.2c_summary'></a>

<div class="alert alert-block alert-success">

### 5.2c Temperature related variables Summary (back to [the beginning of 5.2c](#section_5.2c))

    
<b>Discard all</b>

   - 'AvgSurfT': 'Average surface skin temperature (K)'

   - 'SoilT_0_10cm': 'Soil temperature (0-10cm) (K)'

   - 'SoilT_10_40cm': 'Soil temperature (10-40cm) (K)'

   - 'SoilT_40_100cm': 'Soil temperature (40-100cm) (K)'

   - 'SoilT_100_200cm': 'Soil temperature (100-200cm) (K)'
     
   - 'Albedo': 'Surface albedo (%)'   

<b>Aggregated features to keep</b>
   
   - 'MJJA_Albedo'
    
</div>



### Finalize feature selection of the temperature related variables 

   - ####  `drop_temperature` - columns to drop
   
   - ####  `keep_temperature` - columns to keep
   

In [ ]:
drop_temperature = [col for col in df_target_feature.columns if col.startswith('AvgSurfT_') or \
                                                              col.startswith('SoilT_0_10cm_') or \
                                                              col.startswith('SoilT_10_40cm_') or \
                                                              col.startswith('SoilT_40_100cm') or \
                                                              col.startswith('SoilT_100_200cm_') or \
                                                              col.startswith('Albedo_')]


keep_temperature = ['MJJA_Albedo']

df_target_feature.drop(drop_temperature, axis=1, inplace=True)

### Sanity check that `drop_temperature` are gone and `keep_temperature` remain

In [ ]:
for col in drop_temperature:
    if col in df_target_feature.columns:
        print(col +' should be removed, double check your procedure.')

for col in keep_temperature:
    if not col in df_target_feature.columns:
        print(col +' should be retained, double check your procedure.')

### Update `drop_columns` and `keep_columns`

In [ ]:
drop_columns.append(drop_temperature)
keep_columns.append(keep_temperature)

---
<a id='section_5.2d'></a>
<div class="alert alert-block alert-info">
    
### 5.2d Soil moisture related (jump to the [5.2d summary](#section_5.2d_summary))

- 'SoilM_0_10cm': 'Soil moisture content (0-10cm) (kg m-2)'
- 'SoilM_10_40cm': 'Soil moisture content (10-40cm) (kg m-2)'
- 'SoilM_40_100cm': 'Soil moisture content (40-100cm) (kg m-2)'
- 'SoilM_100_200cm': 'Soil moisture content (100-200cm) (kg m-2)'
- 'SoilM_0_100cm': 'Soil moisture content (0-100cm) (kg m-2)'
- 'SoilM_0_200cm': 'Soil moisture content (0-200cm) (kg m-2)'
- 'RootMoist': 'Root zone soil moisture (kg m-2)'
  
The following variables overlap with above soil moisture variables, so we directly drop these columns.
- 'SMLiq_0_10cm': 'Liquid soil moisture content (0-10cm) (kg m-2)'
- 'SMLiq_10_40cm': 'Liquid soil moisture content (10-40cm) (kg m-2)'
- 'SMLiq_40_100cm': 'Liquid soil moisture content (40-100cm) (kg m-2)'
- 'SMLiq_100_200cm': 'Liquid soil moisture content (100-200cm) (kg m-2)'
- 'SMAvail_0_100cm': 'Soil moisture availability (0-100cm) (%)'
- 'SMAvail_0_200cm': 'Soil moisture availability (0-200cm) (%)'
    
</div>

Back to [Table of contents](#section_0)

### Check the feature redundancy

`SoilM_0_10cm` strongly correlates with soil moisture from deeper levels, so there is no need to keep them all. Let's keep `SoilM_0_10cm` for now and discard soil moisture from deep levels. 

`SoilM_0_10cm` is also correlated with `RootMoist`, but the correlation magnitude is somewhat modest, so we keep them both for now. 

In [ ]:
utils.plot_corr_climate(df_target_feature, \
                       ['SoilM_0_10cm', 'SoilM_10_40cm'], \
                       ['SoilM_0_10cm', 'SoilM_40_100cm'], \
                       ['SoilM_0_10cm', 'SoilM_100_200cm'], \
                       ['SoilM_0_10cm', 'SoilM_0_100cm'], \
                       ['SoilM_0_10cm', 'SoilM_0_200cm'], \
                       ['SoilM_0_10cm', 'RootMoist'])

### Check the redundancy between `SoilM_0_10cm` and `SWdown`, `EF`, and `Rainf`

`No concerning strong correlation, so it's okay to keep SoilM_0_10cm`

In [ ]:
utils.plot_corr_climate(data_filled,\
                       ['SoilM_0_10cm', 'SWdown'], \
                       ['SoilM_0_10cm', 'EF'], \
                       ['SoilM_0_10cm', 'Rainf'])

In [ ]:
utils.plot_corr_yield_current_climate(df_target_feature, \
                                     ['SoilM_0_10cm', 'RootMoist'], \
                                     climate_variable_dict)

In [ ]:
utils.plot_corr_climate(data_filled, \
                       ['ESoil', 'RootMoist'], \
                       ['ESoil', 'AvgSurfT'], \
                       ['ESoil', 'Albedo'], \
                       ['RootMoist', 'AvgSurfT'], \
                       ['AvgSurfT', 'Albedo'], \
                       ['SWdown', 'AvgSurfT'])

### The above correlation heatmaps suggest a very constant positive correlation between `yield` and `SoilM_0_10cm`, and between `yield` and `RootMoist`. 

How does the seasonal cycle of the `SoilM_0_10cm` and `RootMoist` look like? Do we need to keep all months? 

As a rough exploration, we visualize the scatter plot between `yield` and `SoilM_0_10cm` in three different months, and similarly we plot the scatter between `yield` and `RootMoist`. We also show the distribution of `SoilM_0_10cm` and `RootMoist`. 

It seems that `RootMoist` possess constant behavior across different months, suggesting a weak seasonal cycle of this variable. And the variaiton most comes from spatial variation in `RootMoist`. In contrast, `SoilM_0_10cm` still has seasonal variation in terms of both its scatter plot with yield and its own distribution in different months. 

So for now we keep `SoilM_0_10cm` across all months, and create an aggregated variable of `RootMoist`

### Aggregation of SoilM_0_10cm and RootMoist

Both variables are highly correlated across months, so we don't want to include all months. 

Let's keep MJJA_SoilM_0_10cm and JFMA_RootMoist

In [ ]:
df_target_feature['JFMA_SoilM_0_10cm'] = df_target_feature[['SoilM_0_10cm_Jan', 'SoilM_0_10cm_Feb', \
                                                           'SoilM_0_10cm_Mar', 'SoilM_0_10cm_Apr']].mean(axis=1)

df_target_feature['MJJA_SoilM_0_10cm'] = df_target_feature[['SoilM_0_10cm_May', 'SoilM_0_10cm_Jun', \
                                                           'SoilM_0_10cm_Jul', 'SoilM_0_10cm_Aug']].mean(axis=1)

df_target_feature['JFMA_RootMoist'] = df_target_feature[['RootMoist_Jan', 'RootMoist_Feb', \
                                                         'RootMoist_Mar', 'RootMoist_Apr']].mean(axis=1)

df_target_feature['MJJA_RootMoist'] = df_target_feature[['RootMoist_May', 'RootMoist_Jun', \
                                                        'RootMoist_Jul', 'RootMoist_Aug']].mean(axis=1)

df_target_feature[['yield', 'JFMA_SoilM_0_10cm', 'MJJA_SoilM_0_10cm', 'JFMA_RootMoist', 'MJJA_RootMoist']].corr()

`Figure: Scatter between yield and SoilM_0_10cm`

In [ ]:
plt.plot(df_target_feature['SoilM_0_10cm_Jan'], df_target_feature['yield'], 'o', markersize = 3, alpha = 0.1)
plt.plot(df_target_feature['SoilM_0_10cm_Apr'], df_target_feature['yield'], 'o', markersize = 3, alpha = 0.1)
plt.plot(df_target_feature['SoilM_0_10cm_Oct'], df_target_feature['yield'], 'o', markersize = 3, alpha = 0.1)
plt.xlabel('SoilM_0_10cm')
plt.ylabel('yield')
plt.title('Jan, Apr, Oct')

`Figure: Scatter between yield and RootMoist`

In [ ]:
plt.plot(df_target_feature['RootMoist_Jan'], df_target_feature['yield'], 'o', markersize = 3, alpha = 0.05)
plt.plot(df_target_feature['RootMoist_Apr'], df_target_feature['yield'], 'o', markersize = 3, alpha = 0.05)
plt.plot(df_target_feature['RootMoist_Oct'], df_target_feature['yield'], 'o', markersize = 3, alpha = 0.05)
plt.xlabel('RootMoist')
plt.ylabel('yield')
plt.title('Jan, Apr, Oct')

`Figure: distribution of SoilM_0_10cm`

In [ ]:
ax = sns.distplot(df_target_feature['SoilM_0_10cm_Jan'], bins = 100)
ax = sns.distplot(df_target_feature['SoilM_0_10cm_Apr'], bins = 100)
ax = sns.distplot(df_target_feature['SoilM_0_10cm_Oct'], bins = 100)
ax.set_xlabel('SoilM_0_10cm')
ax.set_ylabel('Density')
ax.set_title('Jan, Apr, Oct')

`Figure: distribution of RootMoist`

In [ ]:
ax = sns.distplot(df_target_feature['RootMoist_Jan'], bins = 100)
ax = sns.distplot(df_target_feature['RootMoist_Apr'], bins = 100)
ax = sns.distplot(df_target_feature['RootMoist_Oct'], bins = 100)
ax.set_xlabel('RootMoist')
ax.set_ylabel('Density')
ax.set_title('Jan, Apr, Oct')

---

<a id='section_5.2d_summary'></a>

<div class="alert alert-block alert-success">

### 5.2d Soil moisture related Summary (back to [the beginning of 5.2d](#section_5.2d))
    
<b>Discard all</b>

   - 'SoilM_10_40cm': 'Soil moisture content (10-40cm) (kg m-2)'

   - 'SoilM_40_100cm': 'Soil moisture content (40-100cm) (kg m-2)'

   - 'SoilM_100_200cm': 'Soil moisture content (100-200cm) (kg m-2)'

   - 'SoilM_0_100cm': 'Soil moisture content (0-100cm) (kg m-2)'

   - 'SoilM_0_200cm': 'Soil moisture content (0-200cm) (kg m-2)'
    
   - 'SMLiq_0_10cm': 'Liquid soil moisture content (0-10cm) (kg m-2)'

   - 'SMLiq_10_40cm': 'Liquid soil moisture content (10-40cm) (kg m-2)'

   - 'SMLiq_40_100cm': 'Liquid soil moisture content (40-100cm) (kg m-2)'

   - 'SMLiq_100_200cm': 'Liquid soil moisture content (100-200cm) (kg m-2)'

   - 'SMAvail_0_100cm': 'Soil moisture availability (0-100cm) (%)'

   - 'SMAvail_0_200cm': 'Soil moisture availability (0-200cm) (%)'
    
   - 'SoilM_0_10cm': 'Soil moisture content (0-10cm) (kg m-2)'
    
   - 'RootMoist': 'Root zone soil moisture (kg m-2)'


<b>Aggregated features to discard</b>

   - 'JFMA_SoilM_0_10cm'
    
   - 'MJJA_RootMoist'

<b>Aggregated features to keep</b>
   
   - 'MJJA_SoilM_0_10cm'
    
   - 'JFMA_RootMoist'
    
</div>



### Feature selection of 5.2d

   - #### `drop_moisture` - columns to drop
    
   - #### `keep_moisture` - columns to keep

In [ ]:
drop_moisture = [col for col in df_target_feature.columns if col.startswith('SoilM_10_40cm_') or \
                                                                col.startswith('SoilM_40_100cm_') or \
                                                                col.startswith('SoilM_100_200cm_') or \
                                                                col.startswith('SoilM_0_100cm_') or \
                                                                col.startswith('SoilM_0_200cm_') or \
                                                                col.startswith('SMLiq_0_10cm_') or \
                                                                col.startswith('SMLiq_10_40cm_') or \
                                                                col.startswith('SMLiq_40_100cm_') or \
                                                                col.startswith('SMLiq_100_200cm_') or \
                                                                col.startswith('SMAvail_0_100cm_') or \
                                                                col.startswith('SMAvail_0_200cm_') or \
                                                                col.startswith('SoilM_0_10cm_') or \
                                                                col.startswith('RootMoist_')] \
                + ['JFMA_SoilM_0_10cm', 'MJJA_RootMoist']

keep_moisture = ['MJJA_SoilM_0_10cm', 'JFMA_RootMoist']

df_target_feature.drop(drop_moisture, axis=1, inplace=True)

### Sanity check that `drop_moisture` are gone and `keep_moisture` remain

In [ ]:
for col in drop_moisture:
    if col in df_target_feature.columns:
        print(col +' should be removed, double check your procedure.')

for col in keep_moisture:
    if not col in df_target_feature.columns:
        print(col +' should be retained, double check your procedure.')

### Update `drop_columns` and `keep_columns`

In [ ]:
drop_columns.append(drop_moisture)
keep_columns.append(keep_moisture)

---
<a id='section_5.2e'></a>

<div class="alert alert-block alert-info">
    
### 5.2e Vegetation and land surface parameters (jump to [5.2e summary](#section_5.2e_summary))

- 'LAI': 'Leaf Area Index (unitless)'
- 'GVEG': 'Green vegetation (fraction)'

- 'ACond': 'Aerodynamic conductance (m s-1)'
- 'CCond': 'Canopy conductance (m s-1)'

- 'RCS': 'Solar parameter in canopy conductance (fraction)'
- 'RCT': 'Temperature parameter in canopy conductance (fraction)'
- 'RCQ': 'Humidity parameter in canopy conductance (fraction)'
- 'RCSOL': 'Soil moisture parameter in canopy conductance (fraction)'
- 'RSmin': 'Minimal stomatal resistance (s m-1)'
- 'RSMacr': 'Relative soil moist. availability control factor (0-1)'
    
</div>

Back to [Table of contents](#section_0)

### We directly skip a bunch of canopy conductance related parameters, including `RCS`, `RCT`, `RCQ`, `RCSOL`, `RSmin`. Since we've retained soil moisture related variables in previous section, it's safe to drop `RSMacr` as well.

### Check the redundancy between `LAI`, `GVEG`, `ACond`, and `CCond`

(1) `LAI` & `GVEG`

`LAI` and `GVEG` are strongly correlated with each other, so we just consider `LAI` as a potential predictor. Note that `LAI` itself also has strong lead-lag autocorrelation across months, so we won't include each month and will do some aggregation to represent `LAI`. Given that we expect `LAI` to reflect the soybean planting and growth performance, it is reasonable to only consider the possible effects of `LAI` before the planting season. 

Therefore, we will discard `GVEG` and only keep an aggregated `LAI` by averaging from current January to April. 

(2) `LAI` & `ACond`

`ACond` is affected by both wind speed and canopy height, so we see a strong correlation between `LAI` and `ACond`. Since we will keep the spring `LAI`, we won't keep `ACond` to avoid redundancy. Given that the `ACond` can be affected by wind speed as well, it could be helpful to include that information during the planting throughout the harvest season. So we will keep an aggregated `ACond` by averaging from current May to October. 

(3) `CCond`

`CCond` has modest correlation with `LAI` and `ACond`, the negative correlation during the planting and growing season is interesting. I don't understand what might have caused the negative correlation. Let's keep `CCond` for now, and we will decide whether to keep selected months or do some aggregations after we check its correlation with yield. 

In [ ]:
utils.plot_corr_climate(df_target_feature, \
                       ['LAI', 'GVEG'], ['LAI', 'CCond'], ['LAI', 'ACond'], ['ACond', 'CCond'])

### Check the correlation with yield

(1) Aggregate `LAI` and `ACond`

Both `LAI` and `ACond` strongly correlates with yield, which is expected especially for `LAI`. Also, both `LAI` and `ACond` are strongly autocorrelated. We will keep `LAI` averaged from Jan to April and `ACond` averaged from May to October. 

(2) Discard `CCond`

Correlation between `CCond` and `yield` is pretty weak, and we don't see a discernable nonlinear relationship between `CCond` and `yield`. Further, as a parameter that affects the water flux between land and atmosphere, `CCond`'s effects may have been partly included in other retained features, such as evaporative fraction or soil moisture. It's safe to discard `CCond`. 

In [ ]:
utils.plot_corr_yield_current_climate(df_target_feature, \
                                     ['LAI', 'ACond', 'CCond'], \
                                     climate_variable_dict)

In [ ]:
plt.plot(df_target_feature['ACond_Apr'], df_target_feature['yield'], 'o')
plt.ylim(0,6)

---
<a id='section_5.2e_summary'></a>

<div class="alert alert-block alert-success">

### 5.2e Vegetation and land surface parameters summary (back to [the beginning of 5.2e](#section_5.2e))

#### Discard all
- 'LAI': 'Leaf Area Index (unitless)'
- 'GVEG': 'Green vegetation (fraction)'
- 'ACond': 'Aerodynamic conductance (m s-1)'
- 'CCond': 'Canopy conductance (m s-1)'
- 'RCS': 'Solar parameter in canopy conductance (fraction)'
- 'RCT': 'Temperature parameter in canopy conductance (fraction)'
- 'RCQ': 'Humidity parameter in canopy conductance (fraction)'
- 'RCSOL': 'Soil moisture parameter in canopy conductance (fraction)'
- 'RSmin': 'Minimal stomatal resistance (s m-1)'
- 'RSMacr': 'Relative soil moist. availability control factor (0-1)'

#### Aggregated 
    
- 'JanToApr_LAI'

- 'MayToOct_ACond'

    
</div>

### Feature engineering the `LAI` and `ACond` aggregation

In [ ]:
LAI_cols   = ['LAI_Jan', 'LAI_Feb', 'LAI_Mar', 'LAI_Apr']
ACond_cols = ['ACond_May', 'ACond_Jun', 'ACond_Jul', 'ACond_Aug', 'ACond_Sep', 'ACond_Oct']

df_target_feature['JanToApr_LAI']   = df_target_feature[LAI_cols].mean(axis=1)
df_target_feature['MayToOct_ACond'] = df_target_feature[ACond_cols].mean(axis=1)

### Check the correlation between yield and the aggregated variables 

In [ ]:
df_target_feature[['yield', 'JanToApr_LAI', 'MayToOct_ACond']].corr()

### Finalize the feature selection of 5.2e

   - #### `drop_land` - columns to drop
   
   - #### `keep_land` - columns to keep 

In [ ]:
drop_land = [col for col in df_target_feature.columns if col.startswith('GVEG_') or \
                                                         col.startswith('LAI_') or \
                                                         col.startswith('ACond_') or \
                                                         col.startswith('CCond_') or \
                                                         col.startswith('RCS_') or \
                                                         col.startswith('RCT_') or \
                                                         col.startswith('RCQ_') or \
                                                         col.startswith('RCSOL_') or \
                                                         col.startswith('RSmin_') or \
                                                         col.startswith('RSMacr_')]
keep_land = ['JanToApr_LAI', 'MayToOct_ACond']

df_target_feature.drop(drop_land, axis = 1, inplace=True)

### Sanity check that `drop_land` is gone and `keep_land` remain

In [ ]:
for col in drop_land:
    if col in df_target_feature.columns:
        print(col +' should be removed, double check your procedure.')

for col in keep_land:
    if not col in df_target_feature.columns:
        print(col +' should be retained, double check your procedure.')

### Update `drop_columns` and `keep_columns`

In [ ]:
drop_columns.append(drop_land)
keep_columns.append(keep_land)

---
<a id='section_5.3'></a>

## 5.3 Exploring the correlation between yield and previous year's climate

Back to [Table of contents](#section_0)

### Thought process

We will search through a few climate variables from previous year's climate that may affect current year's yield based on our domain knowledge. Since land has a longer memory compared to atmosphere, here we focus on possible impacts of previous rain/snow/soil moisture conditions. 

Particularly, we explore following variables

- Snowf, SnowDepth, SnowFrac and Rainf from previous winter (Nov and Dec)

- SoilM_0_10cm and RootMoist from previous year 

- Accumulated Rainf from the past year

### Approach 

Consider Yield (lat, lon, year) with year from 1981 to 2016. 

For each year, obtain the lat/lon corresponding varialbe from the previous year. If the value is missing from the previous year, fill the NaN with the interpolation method. 


#### Create `current_yield` to store yield data and their lat, lon, time dimension

In [ ]:
current_yield= df_target_feature[['lat', 'lon', 'year', 'yield']].reset_index(drop=True)
current_yield.head()

#### Obtain climate variables with one year lead  

Here, we will use the defined function `fetch_NLDAS` to obtain `Rainf` from 1980 to 2015, then create a column in `current_yield_lead_climate` to make sure each grid point of yield data in 1981 to 2016 has a matched climate condition from last year. 

In [ ]:
print(inspect.getdoc(utils.fetch_NLDAS))

#### `lead_NLDAS` stores all NLDAS monthly climate variables from 1980 to 2015. Note that this dataframe would still contain NaN values. 

In [ ]:
# parameters needed for fetch_NLDAS
month_number = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_suffix = ['_Jan', '_Feb', '_Mar', '_Apr', '_May', '_Jun', \
                '_Jul', '_Aug', '_Sep', '_Oct', '_Nov', '_Dec']

lead_NLDAS   = utils.fetch_NLDAS(path_climate, 1980, 2015, month_number, month_suffix)

In [ ]:
lead_NLDAS.head()

In [ ]:
lead_NLDAS['year'] = lead_NLDAS['year'].astype(int)

#### Create a new column `Total_Rainf` in lead_NLDAS for annually accumulated Rainf

<div class="alert alert-block alert-danger">
Note that to make sure the sum of NaN also yield NaN (instead of zero), should set min_count = 1 when do the sum. 
See discussion https://stackoverflow.com/questions/33448003/sum-across-all-nans-in-pandas-returns-zero
</div>

In [ ]:
Rainf_cols = [sub.replace('_', 'Rainf_') for sub in month_suffix]
lead_NLDAS['AnnualTotal_Rainf'] = lead_NLDAS[Rainf_cols].sum(axis=1, min_count=1)
lead_NLDAS['SepToDecTotal_Rainf'] = lead_NLDAS[['Rainf_Sep', 'Rainf_Oct', \
                                                'Rainf_Nov', 'Rainf_Dec']].sum(axis=1, min_count=1)

In [ ]:
lead_NLDAS['NovDecTotal_Snowf']     = lead_NLDAS[['Snowf_Nov', 'Snowf_Dec']].sum(axis=1, min_count=1)

lead_NLDAS['NovDecTotal_SnowDepth'] = lead_NLDAS[['SnowDepth_Nov', 'SnowDepth_Dec']].sum(axis = 1, min_count = 1)

RootMoist_cols = [sub.replace('_', 'RootMoist_') for sub in month_suffix]

lead_NLDAS['AnnualMean_RootMoist'] = lead_NLDAS[RootMoist_cols].sum(axis = 1, min_count = 1)

lead_NLDAS['NovDecMean_RootMoist'] = lead_NLDAS[['RootMoist_Nov', 'RootMoist_Dec']].sum(axis = 1, min_count = 1)

#### Create a new column `Total_Rainf` in `current_yield_lead_climate` to make sure everything matches

In [ ]:
current_yield_lead_climate = []

lead_climate_list = ['AnnualTotal_Rainf', \
                     'SepToDecTotal_Rainf',\
                     'NovDecTotal_Snowf', \
                     'NovDecTotal_SnowDepth', \
                     'NovDecMean_RootMoist', \
                     'AnnualMean_RootMoist']

for year in range(1981, 2017):
    
    # ---- Read in the current year's yield data
    yield_this_year = current_yield[current_yield['year']== year].reset_index(drop=True)
    
    # ---- Obtain the total counts of records from this year
    yield_this_year_counts = len(yield_this_year)
    
    for lead in lead_climate_list:
        # ---- Create a new column in current_yield_lead_climate to store the lead climate variable
        yield_this_year['Lead_' + lead] = ''

        # ---- Loop through the grids
        for grid in range(0, yield_this_year_counts):
            lat = yield_this_year['lat'][grid]
            lon = yield_this_year['lon'][grid]
            
            lead_var    = lead_NLDAS[lead_NLDAS['year'] == (year - 1)][['lat', 'lon', lead]]
            lead_grid   = (lead_var['lat'] == lat) & (lead_var['lon'] == lon)
            lead_series = lead_var[lead_grid].reset_index(drop=True)
    
            # ---- If the corresponding location from the previous year has missing value, 
            # ---- set empty to the new column
            if lead_series.empty:
                yield_this_year['Lead_' + lead][grid] = ''
            else:
                yield_this_year['Lead_' + lead][grid] = lead_series[lead][0]    
            
            # ---- Deal with the missing value
            if yield_this_year['Lead_' + lead].isnull().values.any():
                yield_this_year['Lead_' + lead].interpolate(method='linear', direction = 'forward', inplace=True)
    
    current_yield_lead_climate.append(yield_this_year)
    
    print('Year '+str(year)+' done ...')
    
df_current_yield_lead_climate = pd.concat(current_yield_lead_climate)

### Reset the index and double check datatypes

<div class="alert alert-block alert-danger">
Make sure that the numeric data columns are in type float. This is especially important if you want to fill the missing values. If the datatype is object, the interpolation method won't work.
</div>

In [ ]:
df_current_yield_lead_climate.reset_index(drop=True, inplace=True)
df_current_yield_lead_climate.dtypes

### Conver the columns from object to float

In [ ]:
for lead in lead_climate_list:
    df_current_yield_lead_climate['Lead_'+lead] = df_current_yield_lead_climate['Lead_'+lead].astype(float)

### Deal with NaN

In [ ]:
for lead in lead_climate_list:
    df_current_yield_lead_climate['Lead_'+lead].interpolate(method='linear', direction='forward', inplace=True)

In [ ]:
df_current_yield_lead_climate.isna().sum()

In [ ]:
corr_current_yield_lead_climate = df_current_yield_lead_climate[['yield', \
                                                                 'Lead_AnnualTotal_Rainf', \
                                                                 'Lead_SepToDecTotal_Rainf',\
                                                                 'Lead_NovDecTotal_Snowf', \
                                                                 'Lead_NovDecTotal_SnowDepth', \
                                                                 'Lead_NovDecMean_RootMoist', \
                                                                 'Lead_AnnualMean_RootMoist']].corr()
                                                                

In [ ]:
corr_current_yield_lead_climate

In [ ]:
plt.figure(figsize=(80,80), linewidth=20)
sns.set(font_scale=5)
res = sns.heatmap(corr_current_yield_lead_climate, annot=True, fmt='.1f', cmap='RdBu_r', vmin=-1, vmax=1)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 100, rotation=90)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 100, rotation=0)
res.xaxis.tick_top() # x axis on top
res.xaxis.set_label_position('top')
res.tick_params(length=0)
res.set_title('Yield vs Lead climate conditions', fontsize=200, fontweight='bold', y =  1.1)

### Add `Lead_AnnualTotal_Rainf` to `df_target_feature`

In [ ]:
keep_columns.append('Lead_AnnualTotal_Rainf')

In [ ]:
df_target_feature['Lead_AnnualTotal_Rainf'] = df_current_yield_lead_climate['Lead_AnnualTotal_Rainf']

In [ ]:
df_target_feature[['yield', 'Lead_AnnualTotal_Rainf']].corr()

---
<a id='section_6'></a>

# 6. Summarize and save the data  

Back to [Table of contents](#section_0)

### Double-check that df_target_feature is a clean dataframe without missing values

In [ ]:
for column in df_target_feature.columns:
    if df_target_feature[column].isnull().values.any():
        print('Column '+column+ ' still has NaN values ...')

In [ ]:
df_target_feature.columns

In [ ]:
df_target_feature.describe()

In [ ]:
final_corr = df_target_feature[['JJA_SWdown', 'JJA_LWnet',
       'JFMA_EF', 'JJA_EF', 'SO_EF', 'JFMA_Rainf', 'MJJ_Rainf', 'JF_Snowf',
       'JJA_ESoil', 'MJJA_Albedo', 'MJJA_SoilM_0_10cm', 'JFMA_RootMoist',
       'JanToApr_LAI', 'MayToOct_ACond', 'Lead_AnnualTotal_Rainf']].corr()

In [ ]:
plt.figure(figsize=(80,80), linewidth=20)
sns.set(font_scale=5)
res = sns.heatmap(final_corr, annot=True, fmt='.2f', cmap='RdBu_r', vmin=-1, vmax=1)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 100, rotation=90)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 100, rotation=0)
res.xaxis.tick_top() # x axis on top
res.xaxis.set_label_position('top')
res.tick_params(length=0)
res.set_title('Yield vs Features', fontsize=200, fontweight='bold', y =  1.1)

<div class="alert alert-block alert-success">
    
## Features: 

- #### 'JJA_SWdown': June to August averaged incoming solar radiaiton at surface
    
    - affect temperature, photosynthesis, and potential evaporation 
    
- #### 'JJA_LWnet': June to August averaged net longwave radiation at surface 
    
    - associated with surface temperature (outgoing longwave radiation) and atmospheric humidity and cloud cover, both affect downward longwave radiation 
    
- #### 'JFMA_EF': January to April averaged evaporative fraction 
    
- #### 'SO_EF': September to October averaged evaporative fraction 
      
- #### 'JJA_EF': June to August averaged evaporative fraction  
    
    - evaporative fraction reflects partitioning of net surface energy to evapotranspiration, which affect temperature and soil water availability
    
- #### 'JFMA_Rainf': January to April averaged precipitation rate
    
    - precipitation prior to the soybean planting season affects soil moisture
    
- #### 'JF_Snowf': January to Feburary averaged snowfall rate 
    
    - snowfall during the beginning of the year could affect soil moisture as well, though the effects might be weak and might vary across space
    
- #### 'JJA_ESoil': June to August averaged evaporation from soil
    
    - evaporation from soil could affect soil temperature and near-surface temperature, and it's also related to soil moisture availability
    
- #### 'MJJ_Rainf': May to July averaged precipitation rate
    
    - precipitation during the planting and growing season, likely have a nonlinear relationship with yield
    
- #### 'MJJA_Albedo': May to August averaged surface albedo 
    
    - albedo is influenced by multiple factors, could affect surface temperature
    
- #### 'MJJA_SoilM_0_10cm': May to August averaged surface soil moisture 
    
    - direct reflection of surface soil wetness 
    
- #### 'JFMA_RootMoist': January to April averaged Root zoom moisture 
    
    - root zone soil moisture does not have strong seasonal cycle in this dataset; instead, it seems to be a variable that reflects spatial heterogeneity  
    
- #### 'JanToApr_LAI': January to April averaged leaf area index
    
    - leaf area index prior to the typical soybean planting season; I expect this variable to reflect spring climate condition to some extent, because you would not expect too much spring vegetation cover if it's too cold and dry, which could also affect the soybean yield in the current year
    
- #### 'MayToOct_ACond': May to October averaged aerodynamic resistance 
    
    - related to both canopy height and wind speed, affect energy and water exchange between land and atmosphere 
     
- #### 'Lead_AnnualTotal_Rainf': Annual total rainfall rate from the previoius year
    
    - previous year's rainfall affect past and current soil moisture
    
</div>

<a id='save_selected_feature'></a>


### Save the dataframe 

Back to [Table of contents](#section_0)

In [ ]:
df_target_feature.to_csv(data_dir+'/US_SoybeanYield_ClimateFeature_1981-2016_v1.csv')